The **Portal API for ArcGIS** allows users to perform administration tasks not in the Portal for ArcGIS website.  

<br/>

### Getting Started
The root of this API begins with a well-defined URL called the portal administration root URL.

*example: https://webadaptor.domain.com/portal/portaladmin*

The **PortalAdminManager** class builds off of a users GIS using this URL pattern and is accessible from the **GIS** when a site is a portal not AGOL.

In [2]:
from arcgis.gis import GIS
username = "admin"
password = "********"
gis = GIS(
    'https://agsapipor1.esri.com/portal', 
    username, 
    password,
    verify_cert=False)
portaladmin = gis.portaladmin

The **PortalAdminManager** class provides the entry way into the Portal Administration API for Portal.  Off the root class, administrators can do the following:

- Create Sites
- Export/Import a Portal Site
- Join Site
- Get the Site's Status
- Unregister a machine

The additional functionality is subdivided into logical groupings:

- **System** - This resource is an umbrella for a collection of system-wide resources for your portal. This resource provides access to the ArcGIS Web Adaptor configuration, portal directories, database management server, indexing capabilities, license information, and the properties of your portal.
- **Security** - The security resource is the root of all the security configurations and operations in the portal. Through this resource, you can change the identity providers and the authentication mode for your portal.
- **Federation** - This resource returns information about the ArcGIS Servers registered with Portal for ArcGIS.
- **Logs** - Logs are records written by various components of the portal. You can query the logs, clean the logs, and edit log settings.

<br/>

#### Logs

Logs provides debugging and other important information for administrators.  Here an administrator can clean, edit, or query the site's logs.

##### Accessing the log class:

In [4]:
logs = portaladmin.logs

##### Example: Query the logs on Portal
From time to time, issues may arise with a local portal, and logs files will hold the key.  Administrators can access the logs using the **query** operation on the API.

In [7]:
import datetime
query = logs.query(
    start_time=datetime.datetime.now() - datetime.timedelta(days=17))

The above example will grab all portal messages and return them back to the administrator for the last **17** days.
<br/>
The longer your Portal for ArcGIS is up, there is a chance that log files can be very large.  If this occurs, users can clean the sites logs by invoking **clean** on the Logs class.

##### Example: Cleaning up log files:

In [9]:
logs.clean()

True

#### Federation

Local Portals rely on ArcGIS Server for hosting and a variety of other functionality.  That is why it is extremely important to know if servers are up and everything is running as expected to ensure a rich WebGIS experience.

##### Accessing the federation class:

In [10]:
federation = portaladmin.federation

##### Example Finding Problem Servers

In [11]:
result = federation.validate_all()
for res in result['serversStatus']:
    if res['status'] != "success":
        print([res['serverId'], res['messages']])   

If a server is down, or has a problem, the example above will print out the error message.  

#### Security

The security resource is the root of all the security configurations and operations in the portal. Through this resource, you can change the identity providers and the authentication mode for your portal.

Here administrators can access infomation about:

- OAuth Application
- Enterprise Groups
- Enterprise Users
- SSL Certificates
- Get Token Information
- Get/Update Site Security Configuration

##### Accessing the security class

In [12]:
security = portaladmin.security

##### Accessing OAuth
OAuth resource contains a set of operations that update the OAuth2-specific properties of registered applications in Portal for ArcGIS.


In [39]:
oauth = security.oauth

##### Token Configuration
This resource represents the token configuration within your portal. Use the set property on the **tokens** operation to change the configuration properties of the token service.

In [16]:
token = security.tokens

##### SSL Certificates

This resource lists all the certificates (self-signed and CA-signed) created for the portal machine. This resource also list the certificates currently configured with the portal. The portal securely stores these certificates inside a key store.

Before you enable SSL on your portal, you need to generate certificates and get them signed by a trusted certificate authority (CA). For your convenience, the portal is capable of generating self-signed certificates that can be used during development or staging. However, it is critical that you obtain CA-signed certificates when standing up a production server.

In order to get a certificate signed by a CA, you need to generate a CSR (certificate signing request) and then submit it to your CA. The CA will sign your certificate request which can then be imported into the server by using the import signed certificate operation.

##### Accessing the SSL Certificates

In [18]:
ssl = security.ssl

Sometimes sites need to have certificates delete, updated, or added .  The **ssl** property will give access to do just that.

##### Example: Delete Certifcate By Name

In [19]:
certificates = ssl.certificates
for cert in certificates:
    if cert['Alias name'].lower() == "expired":
        cert.delete()
        break

#### Enterprise Groups

The groups resource is an umbrella for operations to manage enterprise groups within the portal. The resource returns the total number of groups in the system.
As an Administrator, you can Search Enterprise Groups, Refresh Group Membership, Get Users Within Enterprise Group, and Get Enterprise Groups For User.

__ *Note: the groups are not groups in Portal rather groups in Active Directory or other similar user products.* __

#### Enterprise Users
The users resource is an umbrella for operations to manage members within Portal for ArcGIS. The resource returns the total number of members in the system.
As an administrator, you can register enterprise accounts in your portal instance by using the Create User operation. When automatic sign-in for users is disabled in the security configuration, registered enterprise accounts can sign in as members of the portal. This gives you full control on all the accounts within a portal instance.

#### Accessing Enterprise Groups

In [23]:
groups = security.groups

#### Accessing Enterprise Users

In [24]:
users = security.enterpriseusers

##### Example Searching Enterprise Groups
This operation searches groups in the configured enterprise group store. You can narrow down the search using the *query* parameter.

In [ ]:
results = groups.search(query="publishers")

### System
This resource provides access to the ArcGIS Web Adaptor configuration, portal directories, database management server, indexing capabilities, license information, and the properties of your portal.

#### Accessing the system class

In [25]:
system = portaladmin.system

#### Database
The database resource represents the database management system (DBMS) that contains all of the portal's configuration and relationship rules. This resource also returns the name and version of the database server currently running in the portal.
You can use the Update Database Account operation to edit the administrative database account that is used by components within the portal to communicate with the database server.

In [30]:
database = system.database

{'adminAccount': 'admin',
 'name': 'gwdb',
 'version': 'psql (PostgreSQL) 9.3.12'}

#### Directories

The directories resource is a collection of directories that are used by the portal to store and manage content. Beginning at 10.2.1, Portal for ArcGIS supports five types of directories:
Content directory—The content directory contains the data associated with every item in the portal.
- **Database** directory—The built-in security store and sharing rules are stored in a Database server that places files in the database directory.
- **Temporary** directory—The temporary directory is used as a scratch workspace for all the portal's runtime components.
- **Index** directory—The index directory contains all the indexes associated with the content in the portal. The indexes are used for quick retrieval of information and for querying purposes.
- **Logs** directory—Errors and warnings are written to text files in the log file directory. Each day, if new errors or warnings are encountered, a new log file is created.
If you would like to change the path for a directory, you can use the edit operation on the directory object.

##### Accessing the directories

In [32]:
directories = system.directories

##### Accessing an individual directory

In [35]:
directory = directories[0]
print(directory.physicalPath)

{"type":"fileStore","provider":"FileSystem","connectionString":"txfL1Ma_fryXmqsmyNt8Zfm4UEuNL79E-qaf8WbGB-8.","isConnectionStringEncrypted":true}


### Indexes

The indexer resource contains connection information to the default indexing service.

The **System** class allows administrators to check on an re-index a portal site.

In [37]:
system.index_status

{'indexes': [{'databaseCount': 41, 'indexCount': 41, 'name': 'users'},
  {'databaseCount': 262, 'indexCount': 262, 'name': 'groups'},
  {'databaseCount': 7091, 'indexCount': 7091, 'name': 'search'}]}

#### Reindexing Users

In [38]:
system.reindex(mode="USER_MODE")

True

### License
Portal for ArcGIS requires a valid license to function correctly. This resource returns the current status of the license.

As of 10.2.1, Portal for ArcGIS enforces the license by checking the number of registered members and comparing it with the maximum number of members authorized by the license. Contact Esri Customer Service if you have questions about license levels or expiration properties.

Starting at 10.5, Portal for ArcGIS enforces two levels of membership for licensing to define sets of privileges for registered members and their assigned roles.

In [41]:
licenses = system.licenses

In [42]:
licenses

Licenses([('maximumRegisteredMembers', 75),
          ('isValid', True),
          ('features',
           [{'currentRegisteredMembers': 0,
             'displayName': 'Portal for ArcGIS',
             'expiration': 1508223600000,
             'isValid': True,
             'level': '1',
             'maximumRegisteredMembers': 25,
             'name': 'esriPortalLvl1',
             'version': '10.5'},
            {'currentRegisteredMembers': 3,
             'displayName': 'Portal for ArcGIS',
             'expiration': 1508223600000,
             'isValid': True,
             'level': '2',
             'maximumRegisteredMembers': 50,
             'name': 'esriPortalLvl2',
             'version': '10.5'}]),
          ('expiration', 1508223600000),
          ('currentRegisteredMembers', 3)])

In addition to basic license management, entitlement licenses can be looked up and updated.

#### Example looking up Portal entitles of ArcGIS Pro

In [43]:
licenses.entitlements(app="arcgisprodesktop")

{'appId': 'arcgisprodesktop',
 'itemId': 'd4a332a0be784dd2be53cec0a35e0859',
 'queue': []}

#### Web Adaptors

The Web Adaptors resource lists the ArcGIS Web Adaptor configured with your portal. You can configure the Web Adaptor by using its configuration web page or the command line utility provided with the installation.

In [44]:
wa = system.web_adaptors

##### Accessing individual adaptors

In [49]:
adaptor = wa.adaptors[0]
print(adaptor.machineIP)

10.49.52.67


#### Languages
 Controls which languages will appear in portal content search results. Use the Update languages operation to modify which languages' content will be available.
 
##### Accessing the languages

In [50]:
lang = system.languages

### FAQ:
1. Who is the portaladmin API designed for?
The portaladmin API is designed for administrators of local portal sites.
2. What is the portaladmin API?
The portaladmin API of the Python API for ArcGIS is a collection of tools that operated on administration tasks not exposed by the Portal for ArcGIS website.
